In [ ]:
%pip install prophet
%pip install plotly
%pip install pandas

In [2]:
# convert a JSON of occupancy from BCT to simple csv file
# split into past and future data, one for training, one for testing the model

import json
import csv

#currently a downloaded json is used. This can be replaced by a direct API call
# i.e. https://data.bayerncloud.digital/api/v4/things/4cf3e23b-ab0d-4cd1-bda3-2e199b2aecdc/dcls_occupancy_rate/

#file_name = './daten/parkplatz-am-walderlebniszentrum-1.json'
file_name = './parkplatz-bad-kohlgrub-schwebebahn.json'

# Read occupancy.json file
with open(file_name, 'r') as file:
    data = json.load(file)

# Extract date and value from each element
rows = []
rows_future = []

for element in data['data']:
    date = element[0]
    value = int(element[1])

    #023-04-27   #2024-09-25

    if date < '2024-04-27 06:40':
        rows.append({'DS': date[:-6], 'Y': value})
    else:
        rows_future.append({'DS': date[:-6], 'Y': value})

# Write data to CSV file
with open('occupancy.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['DS', 'Y'])
    writer.writeheader()
    writer.writerows(rows)

with open('occupancy_future.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['DS', 'Y'])
    writer.writeheader()
    writer.writerows(rows_future)


In [ ]:
#get weather data from brightsky.dev for a certain location and time period

import requests
import csv
from datetime import datetime, timedelta

# Define the latitude and longitude
filename = 'parkplatz-bad-kohlgrub-schwebebahn-1'
lon = 11.04922
lat = 47.66091

#filename = 'parkplatz-am-walderlebniszentrum-1'
#lon = 10.69391
#lat = 47.55822 

# Define the start and end dates
start_date = datetime(2023, 4, 1)
end_date = datetime.now()

def assign_condition_value(condition):
    if condition is None:
        return 50
    condition = condition.lower()

    if condition == 'dry':
        return 0
    elif condition == 'fog':
        return 25
    elif condition == 'sleet':
        return 50
    elif condition == 'rain':
        return 80
    elif condition == 'snow':
        return 85
    elif condition == 'hail':
        return 95
    elif condition == 'thunderstorm':
        return 100
    else:
        return 50 # best guess...


# Create a list to store the weather data
weather_data = []

# Iterate over each hour between the start and end dates
current_date = start_date
while current_date <= end_date:
    # Format the date in the required format
    date_str = current_date.strftime('%Y-%m-%d')

    # Construct the API URL
    url = f'https://api.brightsky.dev/weather?lat={lat}&lon={lon}&date={date_str}'
    print(url)

    # Make the API request
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # loop through the hourly data and extract the required fields
        for hour in data['weather']:
            # Extract the temperature, sunshine and condition
            timestamp = hour['timestamp']
            temperature = hour['temperature']
            sunshine = hour['sunshine']
            condition = assign_condition_value(hour['condition'])

            # Append the data to the weather_data list
            weather_data.append([timestamp, temperature, sunshine, condition])

    # Increment the current date by 1 hour
    current_date += timedelta(days=1)

# Define the CSV file path
csv_file = './weather_' + filename + '.csv'

# Write the weather data to the CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Timestamp', 'Temperature', 'Sunshine', 'Condition'])
    writer.writerows(weather_data)

print(f'Weather data has been written to {csv_file}.')
